In [1]:
import csv
import json
from typing import TypedDict
from collections import defaultdict

In [2]:
file = "../../data/toflit18_all_flows.csv"

In [3]:
class DataPoint(TypedDict):
    product: str 
    import_dk: int
    import_dkb: int
    export_dk: int
    export_dkb: int

In [4]:
report_dk = defaultdict(dict)
report_others = defaultdict(dict)

with open(file, "r") as muerte:
    reader = csv.DictReader(muerte)
    for i, row in enumerate(reader):
        if row["year"] == "1789":
            if row["export_import"] == "Imports" and row["partner"] == "Dunkerque":
                if row["product_RE_aggregate"]:
                    value = float(row["value"]) if row["value"] else 0.0
                    if row["customs_region_grouping"] not in report_others[row["product_RE_aggregate"]]:
                        report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]] = {}
                    report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]]["product"] = row["product_RE_aggregate"]
                    report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]]["region"] = row["customs_region_grouping"]
                    report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]]["import"] = report_dk[row["product_RE_aggregate"]].get("import", 0) + value

            if row["customs_region_grouping"] == "Flandre" and row["customs_office"] in ['Dunkerque', 'Dunkerque-Basseville']:
                if row["product_RE_aggregate"]:
                    value = float(row["value"]) if row["value"] else 0.0
                    report_dk[row["product_RE_aggregate"]]["product"] = row["product_RE_aggregate"]
                    if row["customs_office"] == 'Dunkerque':
                        if row["export_import"] == "Exports":
                            report_dk[row["product_RE_aggregate"]]["export_dk"] = report_dk[row["product_RE_aggregate"]].get("export_dk", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[row["product_RE_aggregate"]]["import_dk"] = report_dk[row["product_RE_aggregate"]].get("import_dk", 0) + value
                    elif row["customs_office"] == 'Dunkerque-Basseville':
                        if row["export_import"] == "Exports":
                            report_dk[row["product_RE_aggregate"]]["export_dkb"] = report_dk[row["product_RE_aggregate"]].get("export_dkb", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[row["product_RE_aggregate"]]["import_dkb"] = report_dk[row["product_RE_aggregate"]].get("import_dkb", 0) + value

In [9]:
all_data = []
for (product, flows) in report_dk.items():
    export_to_fr = sum([dp['import'] for (region,dp) in report_others[product].items()])
    export_to_colonies = flows.get("export_dk", 0)
    total = flows.get("import_dk",0)

    all_data.append({"value":total - export_to_fr - export_to_colonies, "product": product, "destination" : "entrepôt"})
    all_data.append({"value":export_to_fr, "product": product, "destination" : "France"})
    all_data.append({"value":export_to_colonies, "product": product, "destination" : "colonies ou re-exp"})

In [10]:
all_data

[{'value': 14934610.600140002,
  'product': 'Subsistances',
  'destination': 'entrepôt'},
 {'value': 15432.399860000001,
  'product': 'Subsistances',
  'destination': 'France'},
 {'value': 4175686.0,
  'product': 'Subsistances',
  'destination': 'colonies ou re-exp'},
 {'value': 5275812.000006,
  'product': 'Matières premières',
  'destination': 'entrepôt'},
 {'value': 10319.999994,
  'product': 'Matières premières',
  'destination': 'France'},
 {'value': 904212.0,
  'product': 'Matières premières',
  'destination': 'colonies ou re-exp'},
 {'value': 80596.0, 'product': 'Drogues', 'destination': 'entrepôt'},
 {'value': 241.0, 'product': 'Drogues', 'destination': 'France'},
 {'value': 5542.0, 'product': 'Drogues', 'destination': 'colonies ou re-exp'},
 {'value': 590607.0, 'product': 'Divers', 'destination': 'entrepôt'},
 {'value': 937.0, 'product': 'Divers', 'destination': 'France'},
 {'value': 1108.0, 'product': 'Divers', 'destination': 'colonies ou re-exp'},
 {'value': 2865507.58014371

In [ ]:
with open("data/report_dunkerque.json", "w") as f:
    json.dump(report_dk, f)

In [ ]:
with open("data/report_other_regions.json", "w") as f:
    json.dump(report_others, f)

In [ ]:
with open("data/export_dunkerque_entreprôt.json", "w") as f:
    json.dump(all_data, f)